In [1]:
!python -m zipfile -e kalp-atis.zip . 

In [2]:
!pip install numpy==1.25.2 librosa 


In [3]:
!ls Heartbeat_Sound/

artifact  extrahls  extrastole	murmur	normal	unlabel




1. **Artifact (Artefakt)**: Tıbbi görüntüleme veya ses kaydı gibi teknolojik süreçlerde, istenmeyen veya yanlış sonuçlara yol açabilen dış etkenler veya hatalar anlamına gelir. Örneğin, elektrokardiyogram (EKG) çekimlerinde hareket artefaktları veya elektrotların yerleştirilmesinden kaynaklanan gürültüler olabilir.

2. **Extrahls (Ekstra Sistol)**: Kalp atışlarının normal döngüsünden farklı olarak meydana gelen erken bir kalp kasılmadır. Normal kalp atışı sırasında ani bir ekstra kasılma ile karakterizedir. Bu durum bazen kalp atışı hissi, göğüs ağrısı veya baş dönmesi gibi semptomlara yol açabilir.

3. **Extrastole (Ekstra Sistol)**: Bu terim de "Extrahls" ile aynı anlama gelir. Kalbin normal atım döngüsünden önce gelen ek bir kasılma olarak tanımlanır.

4. **Murmur (Üfleme)**: Kalp seslerinden biri olan ve genellikle anormal kan akışı nedeniyle oluşan bir ses. Üfleme, genellikle stetoskopla dinlendiğinde duyulabilir ve kalbin normal seslerinden farklı bir ses olarak tanımlanır.



In [4]:
#Veri setini temizle

import os
import shutil
from glob import glob

files = ["extrastole","murmur","normal","unlabel"]

dataset_files = glob("Heartbeat_Sound/*")

print(f"dataset_files = {dataset_files}")

for file in dataset_files:
    filename = file.split(os.sep)[-1]
    
    if not filename in files:
        print(f"Dosya Siliniyor.. {file}")
        shutil.rmtree(file,ignore_errors=True)



dataset_files = ['Heartbeat_Sound/extrahls', 'Heartbeat_Sound/unlabel', 'Heartbeat_Sound/normal', 'Heartbeat_Sound/artifact', 'Heartbeat_Sound/murmur', 'Heartbeat_Sound/extrastole']
Dosya Siliniyor.. Heartbeat_Sound/extrahls
Dosya Siliniyor.. Heartbeat_Sound/artifact


In [5]:
dataset_files = glob("Heartbeat_Sound/*")
for file in dataset_files:
    print(f"{file} : {len(os.listdir(file))}")

Heartbeat_Sound/unlabel : 247
Heartbeat_Sound/normal : 351
Heartbeat_Sound/murmur : 129
Heartbeat_Sound/extrastole : 46


In [6]:
import numpy as np
import librosa 


def load_files(files,duration = 10, sr = 22050):
    mfccs = 0
    input_length = sr*duration
    data = []
    for sound_file in files:
        
        X, sr = librosa.load(sound_file,sr = sr, duration = duration)
        dur = librosa.get_duration(y = X, sr = sr)
        
        if round(dur) < duration:
            #print(f"Fixed {sound_file}")
            y = librosa.util.fix_length(X,size = input_length)
            
        mfccs = np.mean(librosa.feature.mfcc(y = X, sr = sr, n_mfcc = 25).T,axis = 0)
        
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)

    return data


In [7]:
dataset_files

['Heartbeat_Sound/unlabel',
 'Heartbeat_Sound/normal',
 'Heartbeat_Sound/murmur',
 'Heartbeat_Sound/extrastole']

In [8]:
extrastole_files = glob(os.sep.join([dataset_files[0],'*.wav']))
extrastole_sounds = load_files(extrastole_files)
extrastole_labels = [2] * len(extrastole_sounds)

In [9]:
unlabel_files = glob(os.sep.join([dataset_files[1],'*.wav']))
unlabel_sounds = load_files(unlabel_files)
unlabel_labels = [-1] * len(unlabel_sounds)

In [10]:
normal_files = glob(os.sep.join([dataset_files[2],'*.wav']))
normal_sounds = load_files(normal_files)
normal_labels = [2] * len(normal_sounds)

In [11]:
murmur_files = glob(os.sep.join([dataset_files[3],'*.wav']))
murmur_sounds = load_files(murmur_files)
murmur_labels = [1] * len(murmur_sounds)

In [12]:
x_data = np.concatenate((normal_sounds,murmur_sounds,extrastole_sounds))
y_data = np.concatenate((normal_labels,murmur_labels,extrastole_labels))

In [14]:
x_test = unlabel_sounds
y_test = unlabel_labels

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8,
random_state=42, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8,
random_state=42, shuffle=True)
y_train = np.array(tf.keras.utils.to_categorical(y_train, 3))
y_test = np.array(tf.keras.utils.to_categorical(y_test, 3))
y_val = np.array(tf.keras.utils.to_categorical(y_val, 3))
test_y=np.array(tf.keras.utils.to_categorical(y_test, 3))

In [19]:
model = Sequential()
model.add(Bidirectional(LSTM(128, dropout=0.05, recurrent_dropout=0.20, return_sequences=True), input_shape = (25,1)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-4), metrics=['acc'])
model.summary()

I0000 00:00:1721851712.663033    1146 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 20:08:32.806442: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/venv/lib/python3.11/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 25, 256)        │       133,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25, 128)        │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 25, 128)        │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25, 128)        │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 25, 64)         │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 25, 64)         │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │         4,803 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 216,259 (844.76 KB)

 Trainable params: 216,259 (844.76 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
history=model.fit(
    x_train, 
    y_train,
    batch_size=3,
    epochs=20,
    validation_data=(x_val, y_val)
)

y_pred = model.predict(x_test, batch_size=5)
scores = model.evaluate(x_test, y_test)

print ("Model Dogruluk (Accuracy): ", round(scores[1]*100),"%")

def preprocess_audio(file_path, duration=10, sr=22050):
    input_length = sr * duration
    process_file = []
    X, sr = librosa.load(file_path, sr=sr, duration=duration)
    dur = librosa.get_duration(y=X, sr=sr)
    if round(dur) < duration:
        y = librosa.util.fix_length(X, input_length)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=25).T, axis=0)
    feature = np.array(mfccs).reshape([-1, 1])
    process_file.append(feature)
    process_file_array = np.asarray(process_file)
    return process_file_array

audio_file_path = 'Heartbeat_Sound/normal/normal__103_1305031931979_D1.wav'
processed_audio = preprocess_audio(audio_file_path)
prediction = model.predict(processed_audio)
predicted_class = np.argmax(prediction)
class_mapping = {0: 'abnormal', 1: 'abnormal', 2: 'normal'}
predicted_label = class_mapping[predicted_class]
# Print the result
print('Ses dosyasi '+audio_file_path+' icin tahmin edilen class / sinif: '+predicted_label)

Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.8617 - loss: 0.3624 - val_acc: 0.8971 - val_loss: 0.2831
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.8777 - loss: 0.3312 - val_acc: 0.8971 - val_loss: 0.2875
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.8876 - loss: 0.3401 - val_acc: 0.8971 - val_loss: 0.2875
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.8951 - loss: 0.2888 - val_acc: 0.8971 - val_loss: 0.3023
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.8621 - loss: 0.3554 - val_acc: 0.8971 - val_loss: 0.2867
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.8621 - loss: 0.3328 - val_acc: 0.8971 - val_loss: 0.2811
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.8802 - loss: 0.3265 - val_acc: 0.8971 - val_loss: 0.2856
Epoch 8/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.8630 - loss: 0.3283 - val_acc: 0.8971 - val_loss: 0.2816
Epoch 9/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.8779